# Exploring how Lichess' players spend their playing time

*Chess addiction has always been a recurring subject online, and while its formal existence should be left to rigourous studies, it has lead me to wonder more generally about how players spent their time playing on Lichess, which is the topic will delve into today.*

The data was aggregated from the January 2023 Lichess rated games (freely available on the [Lichess database](https://database.lichess.org)). Games shorter than 4 plies were filtered out. For each account (which from now on I will consider equivalent to one user), I kept their their username, as well as for each Time Control (TC):
*   Number of games
*   Average rating
*   Approximate time spent (in seconds), which is the total time they spent playing this TC, with each game duration computed from the starting clock and increment, [using the fhe Lichess' formula](https://lichess.org/faq#time-controls) to differenciate between blitz/bullet/etc: (**clock initial time in seconds) + 40 × (clock increment)**.
*   Real time spent (in seconds),  which is the total time they spent playing this TC, based on the last time left on the clock at the end of the game, and taking into account increment of course. It does not take into account the use of the `+15s` button, which was considered negligible.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Define global style of the plt for all graphs
plt.style.use("dark_background")
plt.rcParams['axes.facecolor'] = "#262421" # "#444445" # darker is "#262421"
plt.rcParams["figure.facecolor"] = "#161612" #"#333" # "#darker is "#161612"
plt.rcParams["grid.color"] = "#fff"
plt.rcParams["grid.linestyle"] = "-."
plt.rcParams["grid.linewidth"] = 0.2

DFT_HEIGHT = 4.8 # inches
DFT_WIDTH = 6.4 # inches

perfs = ["ultrabullet", "bullet", "blitz", "rapid", "classical"]
colors = [ "#1f77b4","#ff7f0e","#2ca02c","#d62728","#9467bd","#8c564b","#e377c2","#7f7f7f","#bcbd22","#17becf"]
COLOR_MAP = { "ultrabullet": "#fdb462", "bullet": "#81b1d2", "blitz": "#fa8174", "rapid":"#b3de69", "classical":"#bfbbd9"}

In [ ]:
dtypes = {"username": "string"}

#for perf in perfs:
#    dtypes[f"{perf}_approximate_time"] = "Int64"
#    dtypes[f"{perf}_real_time"] = "Int64"
#    dtypes[f"{perf}_avg_rating"] = "Int64"

def restrict_to_perf(perf: str, with_error = True):
    l = ["username", f"{perf}_games", f"{perf}_avg_rating",f"{perf}_approximate_time", f"{perf}_real_time"]
    if with_error:
        l.append(f"{perf}_relative_error")
    return l

#df = pd.read_csv("time-spent.csv",dtype=dtypes)
df = pd.read_csv("time-spent-2023-01.csv",dtype=dtypes)
df.dtypes

In [ ]:
df_time = df.copy()

def convert(c):
    df_time[c] = df_time[c].apply(lambda x: pd.to_timedelta(x,unit="S"))

for perf in perfs:
    convert(f"{perf}_approximate_time")
    convert(f"{perf}_real_time")
df_time

Here is an extract from the data

In [ ]:
print(df_time[df_time["bullet_games"] > 2].iloc[:2][restrict_to_perf("bullet",False)].to_markdown())

|    | username            |   bullet_games |   bullet_avg_rating | bullet_approximate_time   | bullet_real_time   |
|---:|:--------------------|---------------:|--------------------:|:--------------------------|:-------------------|
|  0 | ange_de_la_mode     |             85 |                1430 | 0 days 01:32:20           | 0 days 02:26:59    |
|  5 | Ankit_khandelwal_07 |            214 |                1316 | 0 days 03:39:00           | 0 days 05:33:30    |

The total data has ~~2,073,933~~ 2,073,569 (removing bots) distinct users!

In [ ]:
df_time.describe()

## Overview

A simple first approach is to check how is time played dispatched between the different time controls.

In [ ]:
def pie_per_column(df, column: str):
    res = []
    for p in perfs:
        res.append(df[column.format(p)].sum())
    return res

games = pie_per_column(df,"{}_games")
approx_time = pie_per_column(df,"{}_approximate_time")
real_time = pie_per_column(df,"{}_real_time")

fig, axs = plt.subplots(ncols=3,figsize=(DFT_WIDTH * 2.5, DFT_HEIGHT))
axs[0].pie(games,labels=perfs,colors=COLOR_MAP.values(),autopct="%d%%")
axs[0].set_title("Number of games per Time Control")
axs[1].pie(real_time,labels=perfs,colors=COLOR_MAP.values(),autopct="%d%%")
axs[1].set_title("Real time played per Time Control")
axs[2].pie(approx_time,labels=perfs,colors=COLOR_MAP.values(),autopct="%d%%")
axs[2].set_title("Approximate time played per Time Control")

As expected the number of games alone is not a good appreciation of how much total time is spent playing. We can also see that overhaul, the approximate time formula is quite close to the truth.

But how is time spent playing distributed among the players?

In [ ]:
def real_time_box_plot(ax, df, perfs):
    data = df[[f"{x}_real_time" for x in perfs]]
    data_dropna = [values.dropna() for (_, values) in data.items()]
    colors = [COLOR_MAP[x] for x in perfs]
    bplot = plt.boxplot(data_dropna,patch_artist=True,sym='',labels=perfs,showmeans=True)
    for patch, color in zip(bplot['boxes'], colors):
        patch.set_facecolor(color)
fig = plt.figure(figsize=(DFT_WIDTH * 8 / 5, DFT_WIDTH))
ax = fig.add_subplot(1,1,1,)#nrows=1,)
datafile = 'user-blog-default.png'
img = plt.imread(datafile)
real_time_box_plot(ax,df,perfs)
print(ax.get_xlim())
xl, xh = ax.get_xlim()
yl, yh = ax.get_ylim()
ax.imshow(img, zorder=0,alpha=0.5,origin="upper",extent=[xl,xh, yl, yh],aspect="auto")
ax.set_ylabel("real time spent playing (s)")
fig.suptitle("Real time spent playing distribution")
fig.set_tight_layout(True) # for the blog post head image

This graph shows us than the very important weight of blitz in the overhaul dataset is not merely explained by the fact that they are more players than, let's rapid players, but also because on the average (triangle in the figure) they each spend more time playing it than other TC. 
Another noteworthy point concerning the mean time spent is that it is quite often higher than the 75 percentile, indicating some very strong outliers which mean in our case super active players.

In [ ]:
df1 = df[restrict_to_perf("bullet",False)].copy()
df1.sort_values("bullet_real_time", ascending=False, inplace=True)
df1.iloc[:20]

Now that we have a first overview of the time spent by time control, it's interesting to consider how in a specific TC the data is distributed.

In [ ]:
fig, axs = plt.subplots(ncols=2,nrows=2,figsize=(DFT_WIDTH * 2, DFT_HEIGHT * 2))
axs = axs.flatten()
def hist(df, ax, data, color,perf,log: bool):
    ax.hist(df[data["column"]].astype("float64"),bins=100,color=color,log=log,label=perf)
    ax.set_title(data["title"])

def perf_hist(df, axs, perf: str,include_avg=False,log=False):
    datas = [{
        "column": f"{perf}_games",
        "title": "number of games",
    }]
    if include_avg:
        datas.append({
       "column": f"{perf}_avg_rating",
       "title": "average rating",
        })
    datas.extend([
    {
        "column": f"{perf}_approximate_time",
        "title": "approximate time",
    },{
        "column": f"{perf}_real_time",
        "title": "approximate time",
    }])
    for (ax, data) in zip(axs, datas):
        hist(df,ax,data,COLOR_MAP[perf],perf,log=log)

perf_hist(df1, axs, "bullet",include_avg=True)
fig.suptitle(f"Bullet distribution by")

The distribution of number of games, and time seem to follow a tail distribution, while the rating distribution is following the expected the normal law. A big surprise compared to the [online rating distribution](https://lichess.org/stat/rating/distribution/bullet) is the prominence of new players that is represented by the big spike around 1500.

Let's see if the tail distribution is shared by the Time Control, and use a **logarithm** y-axis this time.

In [ ]:
fig, axs = plt.subplots(ncols=3,nrows=len(perfs),figsize=(DFT_WIDTH * 2.5, DFT_HEIGHT * len(perfs)))

for perf, axs_row in zip(perfs, axs):
    perf_hist(df, axs_row,perf,log=True)
    axs_row[0].legend()
fig.suptitle(f"Distribution by Time Control of",y=0.92,size="x-large")

While this overview clearly demonstrate all follow the same law, the lack of common scale make it difficult to compare them.

In [ ]:
distribs = [np.histogram(df[f"{perf}_real_time"].dropna(),bins=10000) for perf in perfs]
distrib_plot = []
for distrib in distribs:
    # we want to compute the average of the bucket
    average_x = []
    for i in range(0,len(distrib[1]) - 1):
        average_x.append((distrib[1][i] + distrib[1][i+1]) / 2)
    # remove last value which is still a bucket edge
    # now go from y,x to x,y
    distrib_plot.append([average_x, distrib[0]])

#fig, ax = plt.subplot()
for perf, plot_line in zip(perfs, distrib_plot):
    plt.loglog(plot_line[0], plot_line[1],color=COLOR_MAP[perf],label=perf)
plt.suptitle("time spent playing distribution")
plt.ylabel("number of players")
plt.xlabel("total real time spent playing (s)")
plt.legend()

TODO: write comment

In [ ]:
# try to see if by excluding the tail, we can get a more normal-like distrib
column = "bullet_real_time"
df[df[column] < df[column].quantile(0.50)][column].hist(bins=100)
# Seems like no, it's a tail-like distrib

In [ ]:
# Compute the relative error of the real time spent, compared to the average computed one
df_avg_error = df.copy()
for perf in perfs:
    df_avg_error[f"{perf}_relative_error"] = (df[f"{perf}_approximate_time"] - df[f"{perf}_real_time"]).abs() / df[f"{perf}_real_time"]

# Restricting to blitz to see all columns
df_avg_error[restrict_to_perf("blitz")]

## How accurate is the approximate game duration 

Let's now look a bit deepe

In [ ]:
avg_errors = [f"{p}_relative_error" for p in perfs]
def filter_relative_error(df, perfs):
    return [df[df[f"{c}_relative_error"] < df[f"{c}_relative_error"].quantile(0.99)][f"{c}_relative_error"] for c in perfs]

def relative_better_boxplot(ax, df, perfs):
    data = filter_relative_error(df, perfs)
    colors = [COLOR_MAP[x] for x in perfs]
    bplot = ax.boxplot(data,patch_artist=True,sym='',labels=perfs)
    for patch, color in zip(bplot['boxes'], colors):
        patch.set_facecolor(color)

    
fig, axs = plt.subplots(ncols=2,nrows=1,figsize=(DFT_WIDTH * 1.5, DFT_HEIGHT),gridspec_kw={"width_ratios": [4, 1]})
relative_better_boxplot(axs[0],df_avg_error,perfs[:-1])
y_label = "absolute relative duration error"
axs[0].set_ylabel(y_label)
relative_better_boxplot(axs[1],df_avg_error,perfs[-1:])
axs[1].set_ylabel(y_label)
fig.suptitle("Game duration approximation limitation")

As we can see the approximate game formula used by Lichess is quite accurate for short time controls, but as the starting time on the clock get longer it is more and more inaccurate (classical had to be split into its own subplot to account for the change in scale). It could be because some classical games are blitzed or finish very quickly, sadly no data is available at the game level.

Let's see what the non-absolute relative error distribution looks like.

In [ ]:
non_abs_rel_error = df.copy()
for perf in perfs:
    non_abs_rel_error[f"{perf}_relative_error"] = (df[f"{perf}_approximate_time"] - df[f"{perf}_real_time"]) / df[f"{perf}_real_time"]


fig, axs = plt.subplots(ncols=2,nrows=3,figsize=(DFT_WIDTH * 2, DFT_HEIGHT * 3))
axs = axs.flatten()

def perf_hist_err(df, ax, perf: str):
    col = f"{perf}_relative_error"
    data = df[ (df[col] < df[col].quantile(0.95)) & (df[col] > df[col].quantile(0.01))][col]
    ax.hist(data,bins=100,color=COLOR_MAP[perf],label=perf)
    ax.set_xlabel("relative error")
    ax.legend()
    #ax.set_xlim(left=df[err].quantile(0.1),right=df[err].quantile(0.99))

for perf, ax in zip(perfs, axs):
    perf_hist_err(non_abs_rel_error, ax,perf)
fig.suptitle()

Interestingly the calculation tends to under-estimate the game duration in short time controls, then start over-estimating it. Note that the outliers have been removed for better visibility. I'm not sure what phenomon create these spikes on the UB (ultrabullet) distribution, but I assume it has to do with the degree of precision we can have from the database which is limited to the second, and since UB games are very short these can add up.

In [ ]:
df_avg_error[[f"{p}_relative_error" for p in perfs]].describe(percentiles=[.5, .75,.90,.95,.99])

In [ ]:
# Investing why the max error is so high

max_avg_error_first = df_avg_error.sort_values("blitz_relative_error", ascending=False)
max_avg_error_first.query("blitz_games>=10")[restrict_to_perf("blitz")][:20]

In [ ]:
# Only show players with more than x games
x = 30
for (perf, color) in zip(perfs, colors):
    more_x_games = df_avg_error.query(f"{perf}_games>{x}").copy()
    print(f"{more_x_games.shape[0]} players with more than {x} {perf} games")
    more_x_games.hist(bins=100,column=f"{perf}_relative_error",color=color)

## Do better players play more?

While correlation is nause causality, it is interesting to see if better players play more than the average. Let's have a first approach

In [ ]:
# Scatter plot, x rating, y time-spent
#for (perf, color) in zip(perfs, COLOR_MAP.values()):
#    df.plot.scatter(x=f"{perf}_avg_rating",y=f"{perf}_real_time",color=color,s=1)

def scatter(df, ax, perf):
    ax.scatter(x=df[f"{perf}_avg_rating"],y=df[f"{perf}_real_time"],color=COLOR_MAP[perf],s=1)
    ax.set_ylabel("real time (s)")
    #ax.set_label(perf)
    ax.set_xlabel("rating")

fig, axs = plt.subplots(ncols=2,nrows=3,figsize=(DFT_HEIGHT * 4, DFT_WIDTH * 2))
axs = axs.flatten()
for ax, perf in zip(axs, perfs):
    scatter(df, ax, perf)
    ax.legend(labels=[perf],markerscale=5,markerfirst=False) # FIXME: couldn't get to make the legend box appear 
# last one is all combined
for perf in perfs:
    scatter(df, axs[-1], perf)
axs[-1].legend(labels=perfs,markerscale=5,markerfirst=False) # FIXME: couldn't get to make the legend box appear 
fig.suptitle("Real time played per rating")

In [ ]:
# Weighted histogram. Total time per rating
fig, axs = plt.subplots(nrows=len(perfs), figsize=(6.4, 4.8 * len(perfs)))
for (perf, color, ax) in zip(perfs, colors, axs):
    ax.hist(df[f"{perf}_avg_rating"],bins=100,color=color,weights=df[f"{perf}_real_time"])
    ax.set_title(f"{perf} rating distribution, weighted by time played")